In [0]:
#Task1

In [0]:


%sh
wget https://s3-eu-west-1.amazonaws.com/csparkdata/ol_cdump.json /temp/ol_cdump.json



--2022-02-28 07:01:43--  https://s3-eu-west-1.amazonaws.com/csparkdata/ol_cdump.json
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.57.19
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.57.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 139669383 (133M) [application/json]
Saving to: ‘ol_cdump.json’

     0K .......... .......... .......... .......... ..........  0%  198K 11m30s
    50K .......... .......... .......... .......... ..........  0% 13.2M 5m50s
   100K .......... .......... .......... .......... ..........  0%  201K 7m39s
   150K .......... .......... .......... .......... ..........  0%  396K 7m10s
   200K .......... .......... .......... .......... ..........  0% 21.3M 5m45s
   250K .......... .......... .......... .......... ..........  0% 29.2M 4m48s
   300K .......... .......... .......... .......... ..........  0%  409K 4m54s
   350K .......... .......... .......... .......... ......

In [0]:
%fs ls file:///databricks/driver/

path,name,size
file:/databricks/driver/conf/,conf/,4096
file:/databricks/driver/preload_class.lst,preload_class.lst,813069
file:/databricks/driver/eventlogs/,eventlogs/,4096
file:/databricks/driver/logs/,logs/,4096
file:/databricks/driver/ol_cdump.json,ol_cdump.json,139669383
file:/databricks/driver/ganglia/,ganglia/,4096
file:/databricks/driver/metastore_db/,metastore_db/,4096


In [0]:
##Task2 Part1
df=spark.read.json("file:///databricks/driver/ol_cdump.json")
df.count()

Out[5]: 148163

In [0]:
##Task2 Part 2 (curating data with not to include in results with empty/null "titles" and/or "number of pages" is greater than 20 and #"publishing year" is after 1950
df1=df.select("title","publish_date","publishers","authors","genres","number_of_pages","name").filter(df.title != "null").filter(df.title != "").filter(df.number_of_pages >20).filter(df.publish_date>1950)
df1.count()

Out[49]: 75439

In [0]:
#Task2 Part3.1 Select all "Harry Potter" books
df2=df1.filter(df1.title.contains('Harry Potter'))
display(df2)

title,publish_date,publishers,authors,genres,number_of_pages,name
Harry Potter and the philosopher's stone,1998,List(Bloomsbury Publishing),"List(List(null, /authors/OL3218098A, null))","List(Juvenile fiction, Juvenile fiction.)",223,null
Harry Potter y la piedra filosofal,2000,List(Emecé Editores),"List(List(null, /authors/OL3218098A, null))","List(Juvenile fiction, Juvenile fiction.)",254,null


In [0]:
#Task2 Part 3.2 Get the book with the most pages
from pyspark.sql.functions import col, asc,desc
display(df1.select("title","number_of_pages").orderBy(df1.number_of_pages.desc()).take(1))



title,number_of_pages
Nihon shokuminchi kenchikuron,48418


In [0]:
#Task2 Part 3.3 Top 5 authors with most written books
from pyspark.sql.functions import explode,col,count
df5=df1.withColumn("authorname",explode("authors.key")).select("authorname","title")
df6=df5.groupBy("authorname").agg(count("title").alias("title_count_perauthor")).orderBy(col("title_count_perauthor").desc()).take(5)
display(df6)

authorname,title_count_perauthor
/authors/OL1224818A,236
/authors/OL4283462A,116
/authors/OL785848A,106
/authors/OL1926829A,80
/authors/OL883775A,75


In [0]:
#Task2 Part 3.4 Top 5 genres with most books
df1.createOrReplaceTempView("tab1")
display(spark.sql("select distinct explode(genres) as genre,count(distinct title) as titlecount from tab1 group by 1 order by 2 desc").take(5))


genre,titlecount
Fiction.,2757
Biography.,1984
Juvenile literature.,1105
Exhibitions.,760
Congresses.,394


In [0]:
#Task2 Part 3.5 Get the avg. number of pages
from pyspark.sql.functions import avg
adf=df1.agg(avg("number_of_pages"))
display(adf)

avg(number_of_pages)
231.19645011201104


In [0]:
#Task2 Part 3.6 Per publish year, get the number of authors that published at least one book
from pyspark.sql.functions import explode,col, count
from pyspark.sql.window import Window
df5=df1.withColumn("authorname",explode("authors.key")).select("authorname","title","publish_date")
df6=df5.groupBy("authorname","publish_date").agg(count("title").alias("title_count_perauthor")).orderBy(col("title_count_perauthor"))
df7=df6.groupBy("publish_date").agg(count("authorname").alias("number_of_authors"))
#df7=df6.filter("authorname =='/authors/OL4740894A'")
display(df7)

publish_date,number_of_authors
3606,1
1953,600
1957,705
1987,1153
1956,638
1958,753
1972,1010
1988,1212
1977,704
1971,1070
